In [ ]:
#################
#### IMPORTS ####
#################

# Arrays
import numpy as np
import cytoolz

# Deep Learning stuff
import torch
import torchvision
import torchvision.transforms as transforms

# Images display and plots
import matplotlib.pyplot as plt

# Fancy progress bars
import tqdm.notebook as tq

# Tensor Network Stuff
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn # Tensor Network library
import quimb

import collections
import opt_einsum as oe
import itertools


In [ ]:
import sys, os
sys.path.insert(0, '../')
# My functions
from TNutils import *


In [ ]:
# DATASET:
mnist = True      # Set False to use bars and stripes
restricted = True # use 14x14 instead of 28x28 if mnist is True
train_size = 2    # Size of the training set

# MPS:
initial_bdim = 16 # Initial bond dimension of MPS

# TRAINING:
period_epochs = 1 # Number of epochs per period
periods = 20      # Number of periods (at the end of eachperiod mps is saved and data in shown)
batch_size = 50   # batch_size
max_bond = 4      # maximum bond dimension during splits

# LR:
lr0 = 0.1         # Initial Learning Rate
momentum = 0.5    # Momentum term
s_factor = 0.01   # How strongly lr decreases during he epochs

In [ ]:
if mnist:
    train_set, test_set = get_data()
    train_set = train_set[:train_size]
    shape = (28,28)
    if restricted:
        train_set = meanpool2d(train_set, shape)
        test_set  = meanpool2d(test_set, shape)
        shape= (14,14)
else:
    train_set = bars_n_stripes(train_size)
    test_set  = []
    shape= (4,4)

# Convert accordingly
_imgs = np.array([tens_picture(img) for img in train_set])

# Initialize MPS
mps = initialize_mps(_imgs.shape[1],bdim=initial_bdim)

# Initialize the cache
img_cache = left_right_cache(mps,_imgs[:])

sav_lr = mps_lr(mps, lr0, momentum, s_factor)
update_wrap = lambda site, div: sav_lr.J(site, div)

In [ ]:
# TRAINING:
nlls, samples = training_and_probing(
    period_epochs,
    periods,
    mps,
    shape,
    train_set,
    _imgs,
    img_cache,
    batch_size,
    lr = sav_lr,
    lr_update = lr_update,
    update_wrap = update_wrap,
    val_imgs = test_set,
    period_samples = 4,
    corrupted_set = None,
    plot = True,
    max_bond = max_bond)
